<a href="https://colab.research.google.com/github/FrustratedBoy420/Project-Exhibition/blob/main/Diabetic_Patient_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FrustratedBoy420/Project-Exhibition

Cloning into 'Project-Exhibition'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 9), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 1.65 MiB | 4.52 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata # Corrected: Added this import

# Step 1: GPU availability check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Step 2: Hugging Face token ko Colab secrets se access karo
# Corrected: Fetching the token securely
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')

# Step 3: Model aur Tokenizer ko load karo
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected: Added token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True, # Recommended for memory efficiency
    load_in_8bit=True, # Recommended for memory efficiency
    token=huggingface_token # Corrected: Added token
)

print("Mistral-7B model and tokenizer loaded successfully.")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Mistral-7B model and tokenizer loaded successfully.


In [6]:
import json

file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"Data se {len(chunks)} chunks mil gaye.")

Data se 638 chunks mil gaye.


In [7]:
from sentence_transformers import SentenceTransformer

# Embedding model ko load karo
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunks ka embeddings banao
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Embeddings successfully created.")
print(f"Pehle chunk ka embedding shape: {embeddings[0].shape}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

Embeddings successfully created.
Pehle chunk ka embedding shape: (384,)


In [8]:
import numpy as np
import faiss

# Embeddings ko float32 format mein convert karo, jo FAISS ke liye zaruri hai
embeddings_np = np.array(embeddings).astype('float32')
dimension = embeddings_np.shape[1]

# FAISS index banao (yeh ek index hai jo fast search mein madad karta hai)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Index ko save karo taki dobara na banana pade
faiss.write_index(index, "diabetes_faiss_index.bin")

print("FAISS index successfully created and saved.")

FAISS index successfully created and saved.


In [9]:
# Yeh code tumhare existing code ke niche aayega
from google.colab import userdata

def get_recommendation_from_doc(user_query, chunks):
    # 1. User ke sawal ka embedding banao
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # 2. Vector database mein sabse relevant chunks dhoondo (top 3)
    k = 3
    distances, indices = index.search(query_embedding_np, k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # 3. Mistral-7B ke liye prompt banao
    prompt = f"""
    [INST]
    Based on the following medical guidelines from the American Diabetes Association, answer the user's question. Make sure your answer is helpful, accurate, and concise.

    Medical Guidelines: {context}

    User Question: {user_query}
    [/INST]
    """

    # 4. Model se jawab generate karvao
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# Ab isko test karo!
user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"Tumhara sawal: {user_query}")
print(f"Model ka jawab: {recommendation}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Model's Recommendation ---
Tumhara sawal: What are the physical activity recommendations for a diabetic patient?
Model ka jawab: According to the American Diabetes Association (ADA), physical activity is an essential part of diabetes management for all people with diabetes, including those with type 1 and type 2 diabetes. Regular exercise can improve cardiovascular fitness, muscle strength, and insulin sensitivity. The ADA recommends at least 150 minutes of moderate-intensity aerobic activity per week or 75 minutes of vigorous-intensity aerobic activity per week. Additionally, muscle-strengthening activities should be done at least two days per week.

   For diabetic patients, it's important to note that certain conditions may contraindicate certain types of exercise or predispose to injury. These include uncontrolled hypertension, pre-existing diabetes, particularly type 2 diabetes, and those at risk for or presenting with gestational diabetes mellitus, retinopathy, neuropathy, o

##RAG TUNING


In [2]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json


In [11]:
# ----------------------------------
# 2. Device Setup
# ----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ----------------------------------
# 3. Load Model & Tokenizer (with quantization fix)
# ----------------------------------
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Changed to load_in_4bit=True
    bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
    bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    token=huggingface_token # Corrected use_auth_token to token
)

print("✅ Mistral-7B model and tokenizer loaded successfully.")

✅ Using device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Mistral-7B model and tokenizer loaded successfully.


In [12]:
# ----------------------------------
# 4. Load Knowledge Base (JSON file)
# ----------------------------------
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"✅ Loaded {len(chunks)} chunks from knowledge base")

# ----------------------------------
# 5. Create / Load FAISS Index
# ----------------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings_np = np.array(embeddings).astype('float32')

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save index
faiss.write_index(index, "diabetes_faiss_index.bin")
print("✅ FAISS index successfully created and saved.")

✅ Loaded 638 chunks from knowledge base


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

✅ FAISS index successfully created and saved.


In [4]:
# ----------------------------------
# 6. RAG Function
# ----------------------------------
def get_recommendation_from_doc(user_query, chunks, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# ----------------------------------
# 7. Test Query
# ----------------------------------
user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

NameError: name 'chunks' is not defined

In [3]:
# Save the model and tokenizer
# model_save_path = "./mistral_rag_model"
# tokenizer_save_path = "./mistral_rag_tokenizer"

# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(tokenizer_save_path)

# print(f"Model saved to: {model_save_path}")
# print(f"Tokenizer saved to: {tokenizer_save_path}")

# Example of how to load in another file:
# from transformers import AutoTokenizer, AutoModelForCausalLM
#
model_load_path = "./mistral_rag_model"
tokenizer_load_path = "./mistral_rag_tokenizer"

loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path)
loaded_model = AutoModelForCausalLM.from_pretrained(model_load_path)

print("Model and tokenizer loaded successfully in the new file.")

Model and tokenizer loaded successfully in the new file.


In [10]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

# Assuming the model and tokenizer are already loaded from the previous cell
# loaded_tokenizer and loaded_model are available

# 1. Load the FAISS index
index_load_path = "diabetes_faiss_index.bin"
loaded_index = faiss.read_index(index_load_path)
print("✅ FAISS index loaded successfully.")

# 2. Load the chunks
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
loaded_chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            loaded_chunks.append(item['text'])
print(f"✅ Loaded {len(loaded_chunks)} chunks from knowledge base")

# 3. Load the embedding model
from sentence_transformers import SentenceTransformer
loaded_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded successfully.")


# 4. Define the RAG Function using loaded components
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()

    return response_text

# 5. Test Query with loaded components
user_query = "What are the precautions for diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

✅ FAISS index loaded successfully.
✅ Loaded 638 chunks from knowledge base


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Embedding model loaded successfully.

--- Model's Recommendation (using loaded components) ---
❓ Question: What are the precautions for diabetic patient?
✅ Answer: 1. Monitor blood glucose levels regularly, especially during hospitalization and prior to elective procedures.
    2. Implement validated protocols for management of dysglycemia, including glucose monitoring, insulin or non-insulin therapy, and hypoglycemia management.
    3. Ensure a personalized approach to diabetes care, including nutrition recommendations and transitions of care.
    4. Establish protocols for prevention and management of diabetes complications, such as diabetic retinopathy, neuropathy, and foot care.
    5. Provide clear communication and education to outpatient healthcare professionals regarding the patient's diabetes management plan and follow-up appointments.
    6. Implement a structured discharge plan tailored to the individual patient, including appropriate durable medical equipment, medications

In [11]:
# 5. Test Query with loaded components
user_query = "What are the precautions for prediabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Model's Recommendation (using loaded components) ---
❓ Question: What are the precautions for prediabetic patient?
✅ Answer: 1. Screening for type 2 diabetes and prediabetes should begin at no later than age 45 for all people, and earlier for those with overweight or obesity and other risk factors such as family history, physical inactivity, and certain ethnicities.
2. Preconception care is essential for pregnant individuals with pre-existing diabetes and those planning a pregnancy. This includes education on the risks of unplanned pregnancies, improved maternal and fetal outcomes, and effective contraception.
3. Lifestyle interventions to prevent and treat obesity are crucial for individuals with obesity in pregnancy and those with lifestyle-related diabetes.
4. Regular monitoring of glycemic goals and effective diabetes self-management education are recommended for all pregnant individuals with diabetes.
5. Regular testing, including A1C, creatinine, and urinary albumin-to-creat

In [17]:
# =========================================
# PART 2 — QLoRA Fine-tuning + RAG Inference (OOM Safe)
# =========================================

# 0) Install dependencies
!pip -q install -U transformers==4.43.3 accelerate bitsandbytes peft trl datasets sentence-transformers faiss-cpu

import os, json, math, torch, numpy as np, faiss
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          TrainingArguments)
from peft import LoraConfig
from trl import SFTTrainer
from sentence_transformers import SentenceTransformer
from google.colab import userdata

# -----------------------------
# 1) Config
# -----------------------------
BASE_MODEL      = "mistralai/Mistral-7B-Instruct-v0.2"
HF_TOKEN        = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret
JSON_PATH       = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
OUTPUT_DIR      = "/content/mistral-diabetes-qlora"
MAX_SEQ_LEN     = 512           # shorter context to save memory
BATCH_SIZE      = 1             # per device batch
GRAD_ACCUM      = 1             # accumulate less → less VRAM
LR              = 2e-4
EPOCHS          = 1             # try 1 epoch first


# -----------------------------
# 2) Load Model & Tokenizer (with quantization)
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# Quantization config (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

# Load model and tokenizer
# We are using the loaded_model and loaded_tokenizer from the previous cell
model = loaded_model
tokenizer = loaded_tokenizer

# Add padding token
tokenizer.pad_token = tokenizer.eos_token


print("✅ Mistral-7B model and tokenizer loaded successfully.")

# -----------------------------
# 3) Load training data
# -----------------------------
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

records = []
for item in raw:
    text = item.get("text", "").strip()
    if not text:
        continue
    instruction = "Learn these diabetes guidelines to answer patient questions accurately."
    records.append({
        "instruction": instruction,
        "input": text,
        "output": "Acknowledged. I will use this guidance to answer future questions."
    })

print(f"Loaded {len(records)} training examples")

dataset = Dataset.from_list(records).train_test_split(test_size=0.05, seed=42)


# -----------------------------
# 4) Prompt formatting
# -----------------------------
def format_example(ex):
    user = f"{ex['instruction']}\n\nGuidelines:\n{ex['input']}\n\nTask: Confirm understanding."
    assistant = ex["output"]
    return f"[INST]\n{user}\n[/INST]\n{assistant}"

def map_to_text(batch):
    texts = [format_example({key: batch[key][i] for key in batch}) for i in range(len(batch['instruction']))]
    return {"text": texts}

train_texts = dataset["train"].map(map_to_text, batched=True, remove_columns=dataset["train"].column_names)
eval_texts  = dataset["test"].map(map_to_text,  batched=True, remove_columns=dataset["test"].column_names)


# -----------------------------
# 5) LoRA config
# -----------------------------
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

# -----------------------------
# 6) Training arguments
# -----------------------------
train_steps_est = math.ceil(len(train_texts) / (BATCH_SIZE * GRAD_ACCUM)) * EPOCHS
print(f"Approx training steps: ~{train_steps_est}")

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    logging_steps=10,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    fp16=True,                  # ✅ use FP16 (less VRAM)
    save_strategy="steps",
    save_steps=50,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    tokenizer=tokenizer,
    train_dataset=train_texts,
    eval_dataset=None,           # eval skipped
    max_seq_length=MAX_SEQ_LEN,
    packing=False,
    dataset_text_field="text",
)

# -----------------------------
# 7) Train + Save
# -----------------------------
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ LoRA adapters saved to:", OUTPUT_DIR)

✅ Using device: cuda
✅ Mistral-7B model and tokenizer loaded successfully.
Loaded 638 training examples


Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Approx training steps: ~606


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argumen

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
10,3.294100
20,2.916700
30,2.690400
40,2.666000
50,2.610700
60,2.787500
70,2.469300
80,2.628100
90,2.462700
100,2.328900


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in ./mistral_rag_model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in ./mistral_rag_model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in ./mistral_rag_model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in ./mistral_rag_model - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in ./mistral_rag_model - will assume that the vocabulary 

✅ LoRA adapters saved to: /content/mistral-diabetes-qlora


In [18]:
# 5. Test Query with loaded components
user_query = "What are the precautions for prediabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")


--- Model's Recommendation (using loaded components) ---
❓ Question: What are the precautions for prediabetic patient?
✅ Answer: 16.4 For individuals with prediabetes, it is recommended to perform an A1C test every 1–3 years if no A1C result is available from the prior 3 months. The most important component of preconception care is the attainment of glycemic goals prior to conception. In addition, effective contraception should be prescribed and used until the individual is prepared and ready to become pregnant. Diabetes - specific counseling should include an explanation of the risks to mother and fetus related to pregnancies associated with diabetes and the ways to reduce risks, including glycemic goal setting, lifestyle and behavioral management, and medical nutrition therapy. Preconception counseling could avert substantial health and associated cost burdens in the offspring.
